# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint
# Import API key
from api_keys import geoapify_key


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("..\output_data\cities.csv")

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port blair,11.5154,95.4723,28.53,82,100,7.62,in,1686868797
1,1,papatowai,-63.8062,170.9551,2.28,90,100,7.96,nz,1686868798
2,2,edinburgh of the seven seas,-68.6152,4.6462,-9.95,100,77,2.34,sh,1686868799
3,3,waitangi,-64.9264,-150.7886,0.54,94,100,8.99,nz,1686868800
4,4,grytviken,-60.4278,174.2745,5.56,73,100,6.29,nz,1686868800
5,5,ilula,-44.2193,-30.4110,11.27,67,100,10.42,gs,1686868801
6,6,udachny,-7.2941,36.1558,30.29,81,44,2.28,tz,1686868802
7,7,del rio,89.4460,105.6996,0.96,98,100,4.75,ru,1686868803
8,8,yellowknife,29.9688,-101.1775,43.61,9,0,3.83,us,1686868804
9,9,olonkinbyen,72.4384,-117.8199,9.20,69,99,0.63,ca,1686868805


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#%%capture --no-display
scatter_plot = city_data_df.hvplot.scatter(x='Lat', y='Lng', c='Humidity', size='Humidity', cmap='Blues', hover_cols=['City'])

# Customize the plot
scatter_plot.opts(width=800, height=600, colorbar=True, title='City Humidity')

# Display the plot
scatter_plot

:Scatter   [Lat]   (Lng,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
"""
My Ideal weather:
    - Temperature between 15 and 25 degrees
    - Humidity between 30 and 60
    - Cloudiness 0
"""
# Narrow down cities that fit criteria and drop any results with null values
# Drop cities that do not fit Ideal Weather
city_data_new_df = city_data_df.loc[(city_data_df['Cloudiness'] == 0) & (city_data_df['Max Temp'] >= 15) & (city_data_df['Max Temp'] <= 25) & (city_data_df['Humidity'] >= 30) & (city_data_df['Humidity'] <= 60)]

# Drop any rows with null values
city_data_new_df = city_data_new_df.dropna()
city_data_new_df.reset_index(inplace=True, drop=True)
# Display sample data
# Print the number of cities
print(f"There are {len(city_data_new_df)} cities that fit my ideal weather conditions.")
city_data_new_df.head()

There are 2 cities that fit my ideal weather conditions.


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,226,sao joao da barra,-27.6250,30.1845,24.40,59,0,5.28,za,1686868979
1,265,kangiwa,-33.4378,141.4329,19.28,59,0,6.07,au,1686869011


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, "Hotel Name", "")

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port blair,in,11.5154,95.4723,82,
1,papatowai,nz,-63.8062,170.9551,90,
2,edinburgh of the seven seas,sh,-68.6152,4.6462,100,
3,waitangi,nz,-64.9264,-150.7886,94,
4,grytviken,nz,-60.4278,174.2745,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "filter": "",
    "bias": "",
    "apiKey": geoapify_key    
}
# Print a messWage to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
# get latitude, longitude from the DataFrame
latitude = 11.5154
longitude = 95.4723
print(f"({latitude}, {longitude})")
# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
params["filter"] = f"circle:{longitude},{latitude},{radius}"
params["bias"] = f"proximity:{longitude},{latitude}"
# print(params)
# Set base URL
base_url = "https://api.geoapify.com/v2/places"


# Make and API request using the params dictionaty
name_address = requests.get(base_url, params=params)
data = name_address.json()
pprint(data)
"""
print(f"Status: {name_address.status_code}, Index: {0}")
# Convert the API response to JSON format


# Grab the first hotel from the results and store the name in the hotel_df DataFrame
try:
    hotel_df.loc[0, "Hotel Name"] = data["features"][0]["properties"]["name"]
except (KeyError, IndexError):
    # If no hotel is found, set the hotel name as "No hotel found".
    hotel_df.loc[0, "Hotel Name"] = "No hotel found"
    
# Log the search results
print(f"{hotel_df.loc[0, 'City']} - nearest hotel: {hotel_df.loc[0, 'Hotel Name']}")

# Display sample data
hotel_df
"""

Starting hotel search
(11.5154, 95.4723)


In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 20,
    "apiKey": geoapify_key    
}
# Print a messWage to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    print(f"({latitude}, {longitude})")
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    print(params)
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    print(f"Status: {name_address.status_code}, Index: {index}")
    # Convert the API response to JSON format
    data = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
(11.5154, 95.4723)
{'categories': 'accommodation.hotel', 'limit': 20, 'apiKey': 'c47995f4089944a8b0062032c4eb79c6', 'filter': 'circle:95.4723,11.5154,10000', 'bias': 'proximity:95.4723,11.5154'}
Status: 200, Index: 0
port blair - nearest hotel: No hotel found
(-63.8062, 170.9551)
{'categories': 'accommodation.hotel', 'limit': 20, 'apiKey': 'c47995f4089944a8b0062032c4eb79c6', 'filter': 'circle:170.9551,-63.8062,10000', 'bias': 'proximity:170.9551,-63.8062'}
Status: 200, Index: 1
papatowai - nearest hotel: No hotel found
(-68.6152, 4.6462)
{'categories': 'accommodation.hotel', 'limit': 20, 'apiKey': 'c47995f4089944a8b0062032c4eb79c6', 'filter': 'circle:4.6462,-68.6152,10000', 'bias': 'proximity:4.6462,-68.6152'}
Status: 200, Index: 2
edinburgh of the seven seas - nearest hotel: No hotel found
(-64.9264, -150.7886)
{'categories': 'accommodation.hotel', 'limit': 20, 'apiKey': 'c47995f4089944a8b0062032c4eb79c6', 'filter': 'circle:-150.7886,-64.9264,10000', 'bias': 'pro

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
# %%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)